In [23]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "aacuser"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div([
        html.A(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                id="darrell-walker-left",
                style={"width": "100px", "height": "auto", "margin-right": "20px"}
            ),
            href="https://www.snhu.edu",
            target="_blank"
        ),

        html.Div([
            html.H1("Grazioso Salvare Dashboard", style={"margin-bottom": "5px", "color": "#2C3E50"}),
            html.P("By: Darrell Walker", style={"font-size": "16px", "font-weight": "bold", "margin-top": "0px", "color": "#1ABC9C"})
        ], style={"text-align": "center", "flex-grow": "1"}),

        html.A(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                id="darrell-walker-right",
                style={"width": "100px", "height": "auto", "margin-left": "20px"}
            ),
            href="https://www.snhu.edu",
            target="_blank"
        ),
    ], style={"display": "flex", "align-items": "center", "justify-content": "space-between", "backgroundColor": "#ECF0F1", "padding": "10px"}),
    
    html.Hr(),
    
    html.Div([
    # Create a flex container for all filters
    html.Div([
        html.Div([
            html.Label("Filter by Animal Type:", style={'font-weight': 'bold', 'color': '#2C3E50'}),
            dcc.Dropdown(
                id='filter-animal-type',
                options=[{'label': i, 'value': i} for i in df['animal_type'].unique()],
                placeholder="Select animal type...",
                multi=True,
                style={'backgroundColor': 'white', 'color': '#2C3E50', 'borderRadius': '5px'}
            ),
        ], style={'width': '30%', 'padding': '10px'}),

        html.Div([
            html.Label("Filter by Outcome Type:", style={'font-weight': 'bold', 'color': '#2C3E50'}),
            dcc.Dropdown(
                id='filter-outcome-type',
                options=[{'label': i, 'value': i} for i in df['outcome_type'].unique()],
                placeholder="Select outcome type...",
                multi=True
            ),
        ], style={'width': '30%', 'padding': '10px'}),

        html.Div([
            html.Label("Search by Breed:", style={'font-weight': 'bold', 'color': '#2C3E50'}),
            dcc.Input(id="search-breed", type="text", placeholder="Type breed name...", 
                      style={'width': '100%', 'height': '24px', 'borderRadius': '5px', 'border': '1px solid #ccc', 'padding': '5px'})
        ], style={'width': '30%', 'padding': '10px'})
    ], style={'display': 'flex', 'justify-content': 'center'})
]),
    
    # Radio Buttons for Rescue Type Filters
    html.Div([
        html.Label("Filter by Rescue Type:", style={'font-weight': 'bold', 'color': '#2C3E50'}),
        html.Div(
            dcc.RadioItems(
                id='rescue-filter',
                options=[
                    {'label': 'Water Rescue', 'value': 'water'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                    {'label': 'Reset', 'value': 'reset'}
                ],
                value='reset',
                labelStyle={'display': 'inline-block', 'margin-right': '20px', 'color': '#2C3E50'}
            ),
            style={'display': 'flex', 'justify-content': 'center', 'backgroundColor': '#ECF0F1', 'borderRadius': '10px', 'padding': '10px'}
        )
    ]),
    
    html.Hr(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        selected_rows=[0],
        row_selectable="single",
        page_size=15,
        style_table={'overflowX': 'auto', 'border': '2px solid #ddd', 'borderRadius': '5px'},
        style_header={'backgroundColor': '#2C3E50', 'color': 'white', 'fontWeight': 'bold', 'textAlign': 'center'},
        style_cell={'textAlign': 'center', 'padding': '5px'},
        style_data={'backgroundColor': '#ECF0F1', 'color': '#2C3E50'},
        style_data_conditional=[
            {'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'},
            {'if': {'state': 'selected'}, 'backgroundColor': '#1ABC9C', 'color': 'white'}
        ]
    ),
    
    html.Br(),
    html.Hr(),
    
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display': 'flex'},
         children=[
        html.Div(id='graph-id', className='col s12 m6'),

        html.Div([
            html.Div(id='selected-animal-name', style={
                'textAlign': 'center',
                'fontWeight': 'bold',
                'fontSize': '20px',
                'marginBottom': '10px',
                'color': '#2C3E50'
            }),
            html.Div(id='map-id', style={'border': '2px solid #1ABC9C', 'padding': '10px', 'borderRadius': '5px'})
        ], className='col s12 m6')
    ])
])
#############################################
# Interaction Between Components / Controller
#############################################    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-animal-type', 'value'),
     Input('filter-outcome-type', 'value'),
     Input('search-breed', 'value'),
     Input('rescue-filter', 'value')]
)
def update_table(animal_type, outcome_type, breed_search, rescue_filter):
    dff = df.copy()

    if animal_type:
        dff = dff[dff['animal_type'].isin(animal_type)]

    if outcome_type:
        dff = dff[dff['outcome_type'].isin(outcome_type)]

    if breed_search:
        dff = dff[dff['breed'].str.contains(breed_search, case=False, na=False)]

    # Apply Rescue Filters
    if rescue_filter == 'water':
        dff = dff[(dff['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
                  (dff['sex_upon_outcome'] == 'Intact Female') &
                  (dff['age_upon_outcome_in_weeks'].between(26, 156))]
    elif rescue_filter == 'mountain':
        dff = dff[(dff['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
                  (dff['sex_upon_outcome'] == 'Intact Male') &
                  (dff['age_upon_outcome_in_weeks'].between(26, 156))]
    elif rescue_filter == 'disaster':
        dff = dff[(dff['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
                  (dff['sex_upon_outcome'] == 'Intact Male') &
                  (dff['age_upon_outcome_in_weeks'].between(20, 300))]

    return dff.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('rescue-filter', "value")]  # Include rescue filter input
)
def update_graphs(viewData, rescue_filter):
    # Convert input data to DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Check if DataFrame is empty or missing the 'breed' column
    if dff.empty or 'breed' not in dff.columns:
        return html.P("No data available for the selected filter.", 
                      style={'textAlign': 'center', 'fontSize': '16px', 'color': 'red'})

    # If no rescue filter is applied (default 'reset'), show only the top 10 breeds
    if rescue_filter == 'reset':
        breed_counts = dff['breed'].value_counts().nlargest(10).reset_index()
    else:
        breed_counts = dff['breed'].value_counts().reset_index()  # Show all filtered breeds

    # Rename columns for clarity
    breed_counts.columns = ['breed', 'count']

    # Generate Pie Chart
    return [
        dcc.Graph(
            figure=px.pie(breed_counts, names='breed', values='count', title='Preferred Breeds')
        )
    ]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        # If no columns are selected, return an empty list (no styles)
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    [Output('map-id', "children"),
     Output('selected-animal-name', "children")],  # Two outputs (map & name)
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map_and_name(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)

    # Default text if no row is selected
    selected_animal_name = "No animal selected"

    # Default map
    default_map = dl.Map(style={'width': '1000px', 'height': '600px'}, center=[30.75, -97.48], zoom=10)

    # Check if any row is selected
    if not selected_rows or len(dff) == 0 or selected_rows[0] >= len(dff):
        return (default_map, selected_animal_name)  # Return as tuple

    # Select the row data
    selected_animal = dff.iloc[selected_rows[0]]

    # Update the name display
    if pd.notnull(selected_animal["name"]):
        selected_animal_name = f"Name: {selected_animal['name']}"
    elif selected_animal['name'] == "":
        selected_animal_name = "No Name"
    else:
        selected_animal_name = "Name: Unknown"

    # Ensure the selected row has valid latitude and longitude
    if pd.isnull(selected_animal["location_lat"]) or pd.isnull(selected_animal["location_long"]):
        return (default_map, selected_animal_name)

    # Create a single marker for the selected item
    marker = dl.Marker(
        position=[selected_animal["location_lat"], selected_animal["location_long"]],
        children=[
            dl.Popup([
                html.H3(f"Animal: {selected_animal['animal_type']}"),
                html.P(f"Breed: {selected_animal['breed']}"),
                html.P(f"Outcome: {selected_animal['outcome_type']}"),
                html.P(f"Age: {selected_animal['age_upon_outcome']}"),
            ])
        ]
    )

    # Create updated map
    updated_map = dl.Map(
        style={'width': '1000px', 'height': '600px'},
        center=[selected_animal["location_lat"], selected_animal["location_long"]], zoom=12,
        children=[
            dl.TileLayer(),
            marker  # Only one marker appears at a time
        ]
    )

    return (updated_map, selected_animal_name)  # Returning a tuple



app.run_server(debug=True)


Dash app running on http://127.0.0.1:13433/
